In [ ]:
import pandas as pd
from datetime import datetime as dt
import datetime
from copy import deepcopy
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import pylab as pl
%matplotlib inline


import gmplot
from geopy.geocoders import OpenCage, Nominatim, GoogleV3

In [3]:
 def GenerateJupyterLink(FileLocation, LinkText):
    FileLocation = FileLocation.replace('\\','/')
    Link = '<a href="%s">%s</a>' % (FileLocation, LinkText +" - "+FileLocation)
    display(HTML(Link))

def printHTML_DF(dataframe):
        display(HTML(dataframe.to_html(escape=False)))


In [4]:
Data= "/Users/meganpolak/Downloads/2016_Yellow_Taxi_Trip_Data-1.csv'"
Output="/Users/meganpolak/Downloads/Output"

In [ ]:


Raw = pd.read_csv(Data)
Raw = Raw[(Raw.pickup_longitude.notnull()) & Raw.dropoff_longitude.notnull()]

Raw = Raw.query('pickup_latitude != 0 or pickup_longitude != 0')
Raw= Raw.query('dropoff_latitude != 0 or dropoff_longitude != 0')

Raw['tpep_pickup_datetime'] = Data['tpep_pickup_datetime'].apply(lambda x : dt.strptime(x, "%m/%d/%y %H:%M"))
Raw['tpep_dropoff_datetime'] = Raw['tpep_dropoff_datetime'].apply(lambda x : dt.strptime(x, "%m/%d/%y %H:%M"))
Raw['PickupMonth'] = Raw['tpep_pickup_datetime'].dt.month
Raw['DropoffMonth'] = Raw['tpep_dropoff_datetime'].dt.month

printHTML_DF(Raw.head(5))

In [ ]:
round_num = 2
Locations = Raw[['pickup_longitude','pickup_latitude']].rename(columns = {'pickup_longitude':'longitude', 'pickup_latitude':'latitude'})
Locations2 = Raw[['dropoff_longitude','dropoff_latitude']].rename(columns = {'dropoff_longitude':'longitude', 'dropoff_latitude':'latitude'})
Locations = pd.concat([Locations,Locations2], ignore_index=True).round(round_num)

Locations = Locations.drop_duplicates()

Locations.to_excel(OutputFolder + "RawLocations.xlsx", index=False)

In [ ]:
q=0
Encoder_OpenCage = OpenCage(api_key ='afa174d328b0405486d75341c3c43a60')
Encoder_Nominatim = Nominatim()
Encoder_Google = GoogleV3()

def MapLocation(x):
    global q
    LocatorString = str(x['latitude'])+","+str(x['longitude'])
    try:
        location = Encoder_Nominatim.reverse(LocatorString)
        if pd.isnull(location.address):
            raise ValueError("Location Not Found")
        print(str(q)+" "+str(location.address))
        q=q+1
        return location.address
    except Exception as e1:
        print(e1)
        try:
            location = Encoder_OpenCage.reverse(LocatorString)[0]
            if pd.isnull(location.address):
                raise ValueError("Location Not Found")
            print(str(q)+" "+str(location.address))
            q=q+1
            return location.address
        except Exception as e2:
            print(e2)
            try:
                location = Encoder_Google.reverse(LocatorString)[0]
                if pd.isnull(location.address):
                    raise ValueError("Location Not Found")
                print(str(q)+" "+str(location.address))
                q=q+1
                return location.address
            except Exception as e3:
                print(e3)
                return None

In [ ]:
Locations['Address'] = Locations.apply(MapLocation, axis=1)
Locations.to_excel(OutputFolder + "LocationMap.xlsx", index=False)

In [ ]:
Locations = pd.read_excel(OutputFolder + "LocationMap.xlsx")

In [ ]:
Locations['LatLong'] = Locations['latitude'].astype(str) + ' , ' + Locations['longitude'].astype(str) 
Locations['Address'] = Locations['Address'].fillna(Locations['LatLong'])
Locations = Locations.drop(columns=['LatLong'])


Locations.loc[(Locations['Address'].str.contains("Terminal", regex=False)) & (Locations['Address'].str.contains("11434", regex=False)),'Address'] = "John F Kennedy International Airport, Bayswater, Queens County, NYC, New York, 11434, United States of America"

Cols = ['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']

for col in Cols:
    Raw[col+str("_round")] = Raw[col].round(round_num)

Raw = pd.merge(Raw, Locations.rename(columns ={'Address':'PickupAddress'}), how='left', left_on=['pickup_latitude_round','pickup_longitude_round'], right_on =['latitude','longitude']).drop(columns =['latitude','longitude'])
Raw= pd.merge(Raw, Locations.rename(columns ={'Address':'DropoffAddress'}), how='left', left_on=['dropoff_latitude_round','dropoff_longitude_round'], right_on =['latitude','longitude']).drop(columns =['latitude','longitude'])

printHTML_DF(Raw.head(5))

In [ ]:
# Question 1
MonthlyPickupCount = Raw[['PickupMonth','tpep_pickup_datetime']].groupby('PickupMonth').count().rename(columns = {'tpep_pickup_datetime':'No. of Pickups'}).reset_index(drop=False) 
MonthlyPickupCount['PickupMonth'] = MonthlyPickupCount['PickupMonth'].apply(lambda x:datetime.date(1900,x,1).strftime('%B'))
MonthlyPickupCount = MonthlyPickupCount.set_index('PickupMonth')
printHTML_DF(MonthlyPickupCount)
plt = MonthlyPickupCount.plot(kind='bar', title="No. of Pickups per Month")
plt.legend(loc=9, bbox_to_anchor=(0.8, -0.15))
print("\nMax Pickups are in the month of " + str(MonthlyPickupCount['No. of Pickups'].idxmax()))

In [ ]:
# Question 2

def bin_times(x):
    
    Time = x.time()
    
    if Time <= datetime.time(10):
        if Time <= datetime.time(4):
            slot = "21:00 - 04:00"
        else:
            slot = "04:00 - 10:00"
    elif Time <= datetime.time(15):
        slot = "10:00 - 15:00"
    elif Time<= datetime.time(21):
        slot = "15:00 - 21:00"
    else:
        slot = "21:00 - 04:00"
    
    return slot

Raw["PickupSlot"] = Raw["tpep_pickup_datetime"].apply(bin_times)
Raw["DropoffSlot"] = Raw["tpep_dropoff_datetime"].apply(bin_times)

MonthlyPickupSlotMaster = Raw[['PickupMonth','PickupSlot','tpep_pickup_datetime']].groupby(['PickupMonth', 'PickupSlot']).count().reset_index(drop=False)
MonthlyPickupSlot = MonthlyPickupSlotMaster.groupby(['PickupMonth']).apply(lambda x: x.loc[x['tpep_pickup_datetime'].idxmax()]).reset_index(drop=True).rename(columns = {'tpep_pickup_datetime':'No. of Pickups', 'PickupSlot':'Max Pickups Slot'})
MonthlyPickupSlot['PickupMonth'] = MonthlyPickupSlot['PickupMonth'].apply(lambda x:datetime.date(1900,x,1).strftime('%B'))
MonthlyPickupSlot = MonthlyPickupSlot.set_index('PickupMonth')
printHTML_DF(MonthlyPickupSlot)


MonthlyPickupSlotMaster = MonthlyPickupSlotMaster.pivot(index="PickupMonth", columns = "PickupSlot", values = "tpep_pickup_datetime").reset_index(drop=False)

MonthlyPickupSlotMaster['PickupMonth'] = MonthlyPickupSlotMaster['PickupMonth'].apply(lambda x:datetime.date(1900,x,1).strftime('%B'))
MonthlyPickupSlotMaster = MonthlyPickupSlotMaster.set_index('PickupMonth')
plt = MonthlyPickupSlotMaster.plot(kind='bar',stacked=True, title="Pickups in given slot")
plt.legend(loc=9, bbox_to_anchor=(0.8, -0.15))

In [ ]:
# Question 3
MonthlyLongest = Raw.groupby('PickupMonth').apply(lambda x: x.loc[x['trip_distance'].idxmax()]).rename(columns = {'trip_distance':'Longest Trip Distance'}).reset_index(drop=True)
MonthlyLongest['PickupMonth'] = MonthlyLongest['PickupMonth'].apply(lambda x:datetime.date(1900,x,1).strftime('%B'))
MonthlyLongest = MonthlyLongest.set_index('PickupMonth')
printHTML_DF(MonthlyLongest[['Longest Trip Distance']])
MonthlyLongest[['Longest Trip Distance']].plot(kind = 'bar', title = "Longest Trip per Month")
print("\nLongest Longest Distance Trip is in the month of " + str(MonthlyLongest['Longest Trip Distance'].idxmax()))

In [ ]:
# Question 4
printHTML_DF(MonthlyLongest[['PickupAddress','DropoffAddress','dropoff_longitude', 'dropoff_latitude', 'fare_amount', 'Longest Trip Distance']])


In [ ]:
# Question 5

MonthlyExpensive = Raw.groupby('PickupMonth').apply(lambda x: x.loc[x['fare_amount'].idxmax()]).rename(columns = {'fare_amount':'Most Expensive Trip'}).reset_index(drop=True)
MonthlyExpensive['PickupMonth'] = MonthlyExpensive['PickupMonth'].apply(lambda x:datetime.date(1900,x,1).strftime('%B'))
MonthlyExpensive = MonthlyExpensive.set_index('PickupMonth')
printHTML_DF(MonthlyExpensive[['Most Expensive Trip']])
MonthlyExpensive[['Most Expensive Trip']].plot(kind = 'bar', title="Max Trip Cost per month")

print("\nHighest Most Expensive Trip is in the month of " + str(MonthlyExpensive['Most Expensive Trip'].idxmax()))

In [ ]:
# Question 6
AirportsList = ["Aloft New York LaGuardia Airport, 100-15, Ditmars Boulevard, North Beach, Queens County, NYC, New York, 11369, United States of America","John F Kennedy International Airport, Bayswater, Queens County, NYC, New York, 11434, United States of America"]
Airports={}
Airports[AirportsList[0]] = "LaGuardia Airport"
Airports[AirportsList[1]] = "JFK Airport"

MonthlyAirportDropoffs = Raw[RawData.DropoffAddress.isin(AirportsList)]
MonthlyAirportDropoffs = MonthlyAirportDropoffs[['PickupMonth','DropoffAddress','tpep_pickup_datetime']].groupby(['PickupMonth','DropoffAddress']).count().reset_index(drop=False).rename(columns={'tpep_pickup_datetime':'No. of Dropoffs'}).sort_values("PickupMonth")

MonthlyMaxAirport = MonthlyAirportDropoffs.groupby(['PickupMonth']).apply(lambda x: x.loc[x['No. of Dropoffs'].idxmax()]).reset_index(drop=True)
MonthlyAirportDropoffsWide = MonthlyAirportDropoffs.pivot(index="PickupMonth", columns = "DropoffAddress", values = "No. of Dropoffs").rename(columns = Airports).reset_index(drop=False)

MonthlyAirportDropoffs['PickupMonth'] = MonthlyAirportDropoffs['PickupMonth'].apply(lambda x:datetime.date(1900,x,1).strftime('%B'))
MonthlyAirportDropoffs = MonthlyAirportDropoffs.set_index('PickupMonth')
printHTML_DF(MonthlyAirportDropoffs)

MonthlyMaxAirport['PickupMonth'] = MonthlyMaxAirport['PickupMonth'].apply(lambda x:datetime.date(1900,x,1).strftime('%B'))
MonthlyMaxAirport = MonthlyMaxAirport.rename(columns = {"No. of Dropoffs":"Maximum No. of Drops"}).set_index('PickupMonth')
print("\nMaximum Airport Drops per Month")
printHTML_DF(MonthlyMaxAirport)

MonthlyAirportDropoffsWide['PickupMonth'] = MonthlyAirportDropoffsWide['PickupMonth'].apply(lambda x:datetime.date(1900,x,1).strftime('%B'))
MonthlyAirportDropoffsWide = MonthlyAirportDropoffsWide.set_index('PickupMonth')
plt = MonthlyAirportDropoffsWide.plot(kind='bar',stacked=True, title="Dropoffs per airport")
plt.legend(loc=9, bbox_to_anchor=(0.8, -0.15))

In [ ]:
# Question 7
MonthlyCommonPaymentMaster = Raw[['PickupMonth','payment_type','total_amount']].groupby(['PickupMonth', 'payment_type']).count().reset_index(drop=False)
MonthlyCommonPayment = MonthlyCommonPaymentMaster.groupby(['PickupMonth']).apply(lambda x: x.loc[x['total_amount'].idxmax()]).reset_index(drop=True).rename(columns = {'total_amount':'No. of Payments', 'payment_type':'Max Payments Type'})
MonthlyCommonPayment2 = Raw[['PickupMonth','payment_type','total_amount']].groupby(['PickupMonth', 'payment_type']).sum().reset_index(drop=False)

MonthlyCommonPayment = pd.merge(MonthlyCommonPayment, MonthlyCommonPayment2, how = 'left', left_on=["PickupMonth","Max Payments Type"], right_on = ["PickupMonth", "payment_type"]).drop(columns = ["payment_type"]).rename(columns = {'total_fare':'Total Amount Paid via Method'})

MonthlyCommonPayment['PickupMonth'] = MonthlyCommonPayment['PickupMonth'].apply(lambda x:datetime.date(1900,x,1).strftime('%B'))
MonthlyCommonPayment = MonthlyCommonPayment.set_index('PickupMonth')

printHTML_DF(MonthlyCommonPayment)

MonthlyCommonPaymentMaster = MonthlyCommonPaymentMaster.pivot(index="PickupMonth", columns = "payment_type", values = "total_amount").reset_index(drop=False)

MonthlyCommonPaymentMaster['PickupMonth'] = MonthlyCommonPaymentMaster['PickupMonth'].apply(lambda x:datetime.date(1900,x,1).strftime('%B'))
MonthlyCommonPaymentMaster = MonthlyCommonPaymentMaster.set_index('PickupMonth')
plt = MonthlyCommonPaymentMaster.plot(kind='bar',stacked=True, title="Payments by Type")
plt.legend(loc=9, bbox_to_anchor=(0.8, -0.15), ncol = 2)

 print(MonthlyPickupSlot)

In [ ]:
# Question 8
MonthlyMaxLocationPickup = Raw[['PickupMonth','PickupAddress','tpep_pickup_datetime']].groupby(['PickupMonth', 'PickupAddress']).count().reset_index(drop=False)
MonthlyMaxLocationPickup = MonthlyMaxLocationPickup.groupby(['PickupMonth']).apply(lambda x: x.loc[x['tpep_pickup_datetime'].idxmax()]).reset_index(drop=True).rename(columns = {'tpep_pickup_datetime':'No. of Pickups', 'PickupAddress':'Max Pickups Location'})
MonthlyMaxLocationPickup['PickupMonth'] = MonthlyMaxLocationPickup['PickupMonth'].apply(lambda x:datetime.date(1900,x,1).strftime('%B'))
MonthlyMaxLocationPickup= MonthlyMaxLocationPickup.set_index('PickupMonth')
printHTML_DF(MonthlyMaxLocationPickup)

MonthlyMaxLocationDropoff = RawData[['DropoffMonth','DropoffAddress','tpep_dropoff_datetime']].groupby(['DropoffMonth', 'DropoffAddress']).count().reset_index(drop=False)
MonthlyMaxLocationDropoff= MonthlyMaxLocationDropoff.groupby(['DropoffMonth']).apply(lambda x: x.loc[x['tpep_dropoff_datetime'].idxmax()]).reset_index(drop=True).rename(columns = {'tpep_dropoff_datetime':'No. of Dropoffs', 'DropoffAddress':'Max Dropoffs Location'})
MonthlyMaxLocationDropoff['DropoffMonth'] = MonthlyMaxLocationDropoff['DropoffMonth'].apply(lambda x:datetime.date(1900,x,1).strftime('%B'))
MonthlyMaxLocationDropoff= MonthlyMaxLocationDropoff.set_index('DropoffMonth')
printHTML_DF(MonthlyMaxLocationDropoff)



In [ ]:
# Question 9
TripsToJFK = deepcopy(Raw[Raw.DropoffAddress == "John F Kennedy International Airport, Bayswater, Queens County, NYC, New York, 11434, United States of America"])
TripsToJFK["TripLength"] = TripsToJFK["tpep_dropoff_datetime"] - TripsToJFK["tpep_pickup_datetime"]

print(TripsToJFK["TripLength"].describe())
TripLengthSeries = TripsToJFK['TripLength'].astype('timedelta64[s]').divide(60)
plt = TripLengthSeries.hist(bins = 50)
pl.suptitle("Histogram of Trip-Length")

In [ ]:
NightHotspots = deepcopy(Raw[Raw.DropoffSlot == "21:00 - 04:00"])
MonthlyNightHotspots = NightHotspots[['DropoffMonth','DropoffAddress','tpep_dropoff_datetime']].groupby(['DropoffMonth', 'DropoffAddress']).count().reset_index(drop=False)
MonthlyNightHotspots= MonthlyNightHotspots.groupby(['DropoffMonth']).apply(lambda x: x.loc[x['tpep_dropoff_datetime'].idxmax()]).reset_index(drop=True).rename(columns = {'tpep_dropoff_datetime':'No. of Dropoffs', 'DropoffAddress':'Max Dropoffs Location'})
MonthlyNightHotspots['DropoffMonth'] = MonthlyNightHotspots['DropoffMonth'].apply(lambda x:datetime.date(1900,x,1).strftime('%B'))
MonthlyNightHotspots= MonthlyNightHotspots.set_index('DropoffMonth')
printHTML_DF(MonthlyNightHotspots)


gmap = gmplot.GoogleMapPlotter(40.7128, -74.0060, 10)
gmap.heatmap(NightHotspots['dropoff_latitude'], NightHotspots['dropoff_longitude'])
gmap.draw(OutputFolder+"NightSpots.html")
GenerateJupyterLink(OutputFolder+"NightSpots.html", "Nightspots ")